<a href="https://colab.research.google.com/github/bezerraluis/Projeto-AgriDecision/blob/main/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Próximos passos
- Fazer um repositório com mais imagens

In [1]:
!git clone https://github.com/bezerraluis/Projeto-AgriDecision

Cloning into 'Projeto-AgriDecision'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 2925 (delta 0), reused 7 (delta 0), pack-reused 2913
Receiving objects: 100% (2925/2925), 78.57 MiB | 15.31 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
# !git clone https://github.com/starly/Soybean_Images_MachineLearning

In [3]:
cd Projeto-AgriDecision/

/content/Projeto-AgriDecision


In [4]:
ls

plant_diseases_classification_using_Resnet.ipynb  Soybean_downy/
README.md                                         Soybean_healthy/
Resnet50.ipynb                                    Soybean_rust/


In [5]:
#!unzip /content/Soybean_Images_MachineLearning/Soybean_downy_train.zip
#!unzip /content/Soybean_Images_MachineLearning/Soybean_rust_train.zip
#!unzip /content/Soybean_Images_MachineLearning/Soybean_healthy_train.zip

unzip:  cannot find or open /content/Soybean_Images_MachineLearning/Soybean_downy_train.zip, /content/Soybean_Images_MachineLearning/Soybean_downy_train.zip.zip or /content/Soybean_Images_MachineLearning/Soybean_downy_train.zip.ZIP.
unzip:  cannot find or open /content/Soybean_Images_MachineLearning/Soybean_rust_train.zip, /content/Soybean_Images_MachineLearning/Soybean_rust_train.zip.zip or /content/Soybean_Images_MachineLearning/Soybean_rust_train.zip.ZIP.
unzip:  cannot find or open /content/Soybean_Images_MachineLearning/Soybean_healthy_train.zip, /content/Soybean_Images_MachineLearning/Soybean_healthy_train.zip.zip or /content/Soybean_Images_MachineLearning/Soybean_healthy_train.zip.ZIP.


In [7]:
import glob

filepaths = glob.glob('/content/Soybean_Images_MachineLearning/*/*.{jpg,jpeg,png}')

In [8]:
sum('rust_train' in f for f in filepaths)

0

In [9]:
sum('healthy_train' in f for f in filepaths)

0

In [10]:
sum('downy_train' in f for f in filepaths)

0

In [11]:
len(filepaths)

0

In [22]:
!rm -r /content/Projeto-AgriDecision/.git

In [23]:
# TODO: Import the model and the preprocess_input function
from keras.applications.resnet50 import ResNet50, preprocess_input

# TODO: Import the ImageDataGenerator class
from keras.preprocessing.image import ImageDataGenerator

# Shape in which all images are going to be reshaped
TARGET_SHAPE = (224, 224, 3)

# TODO: Initialize the data generator class 
datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                             validation_split=0.2)

# TODO: Create the training and validation generators using the method flow_from_directory
train_gen = datagen.flow_from_directory('/content/Projeto-AgriDecision', 
                                        target_size=TARGET_SHAPE[:2], 
                                        class_mode='sparse',
                                        subset='training')

valid_gen = datagen.flow_from_directory('/content/Projeto-AgriDecision', 
                                        target_size=TARGET_SHAPE[:2], 
                                        class_mode='sparse',
                                        subset='validation')

Found 2173 images belonging to 3 classes.
Found 541 images belonging to 3 classes.


In [24]:
train_gen.class_indices

{'Soybean_downy': 0, 'Soybean_healthy': 1, 'Soybean_rust': 2}

In [25]:
sum(train_gen.classes==1)

1817

In [26]:
sum(train_gen.classes==0)

188

In [27]:
sum(train_gen.classes==2)

168

In [28]:
train_gen.class_indices

{'Soybean_downy': 0, 'Soybean_healthy': 1, 'Soybean_rust': 2}

In [36]:
sum(valid_gen.classes==0)

46

In [34]:
sum(valid_gen.classes==1)

454

In [37]:
sum(valid_gen.classes==2)

41

In [29]:
# 1. TODO: Import any additional packages
from keras.models import Sequential
from keras.layers import Flatten, Dense, GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50

# 2. TODO: Initialize base model
base_model = ResNet50(include_top=False, input_shape=TARGET_SHAPE)

# 3. TODO: Freeze layers from the base model
for layer in base_model.layers:
    layer.trainable=False
    
# 4. TODO: Add Fully connected layer to the base model
model = Sequential([base_model,
                    GlobalAveragePooling2D(),
                    Dense(1024, activation='relu'),
                    Dense(3, activation='softmax')])

In [41]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping

# define roc_callback, inspired by https://github.com/keras-team/keras/issues/6050#issuecomment-329996505
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.keras.metrics.AUC(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value




In [42]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=1e-4), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy',auc_roc,])

In [43]:
model.fit_generator(train_gen, train_gen.n//train_gen.batch_size, epochs=3,
                    validation_data=valid_gen)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3


AttributeError: ignored

In [32]:
model.fit_generator?